# 1st Random forest 
## public : 0.7162478111 / private : 0.7036919758

GH_Def_new를 바탕으로 모델링 시행.

하지만 중간 path 미실행으로 전처리 오류 -> path 실행 할 것.

log_loss: 1.0029295122124835 높은 편. 어떻게 하면 줄일 수 있을까?

## develop 방향 :

age, work_year -> begin_month 반영하여 처리 예정

scaling 실행 예정(일단 MinMaxScaler)


## New Def(GH)

지난번 만든 함수에는 user_list를 만들어서 data가 일부 소실되는 바람에 submission 값도 소실되는 결과가 있었음.

따라서 이번에는 ID 중복을 drop하지 않는 방향으로 진행해 볼 것임

또한, occyp_type을 미리 처리해주고 ID에 occyp_type을 추가하는 시도를 해볼 것임.

+ 함수를 처리하는 중 print로 data를 확인하는 부분이 있는데, 이부분을 color 변경을 통해서 주목성을 높임
  

## 필요한 라이브러리 호출

In [3]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import warnings
from sklearn.preprocessing import OrdinalEncoder


warnings.filterwarnings('ignore')

## 데이터 전처리 과정

In [14]:
# 경로 설정 (dataset을 저장하는 경로임.)

submission = pd.read_csv(r'C:\Users\user\Desktop\code\[유비온]신용카드 사용자 연제 예측\235713_신용카드 사용자 연체 예측 AI 경진대회_data (1)\open\sample_submission.csv')
train = pd.read_csv(r'C:\Users\user\Desktop\code\[유비온]신용카드 사용자 연제 예측\235713_신용카드 사용자 연체 예측 AI 경진대회_data (1)\open\train.csv')
test = pd.read_csv(r'C:\Users\user\Desktop\code\[유비온]신용카드 사용자 연제 예측\235713_신용카드 사용자 연체 예측 AI 경진대회_data (1)\open\test.csv')

In [15]:
#쓸모없는 칼럼 삭제
for df in (train,test):
    df.drop(['FLAG_MOBIL'], axis=1, inplace=True)   
    df.drop(columns=['index'], inplace=True)
    
del df

### 1. 나이, 고용일자를 이용하여 파생변수 형성

In [16]:
# 1st

def year_month_total(df_1, df_2, column, column_name):
    """[나이, 고용연수 파생변수를 생성하기 위한 함수입니다.]

    Args:
        df_1 ([DataFrame]): [데이터 프레임 이름을 입력해주세요]
        df_2 ([DataFrame]): [데이터 프레임 이름을 입력해주세요]
        column ([생성할 컬럼의 기준 컬럼 명]): [생성할 컬럼의 기준 컬럼 명을 입력해주세요]
        column_name ([생성할 컬럼의 이름]): [연단위, 연+월단위로 표시된 파생변수 이름입니다.]
    """
    # 변수 설정
    
    df_list = []
    df_list.append(df_1)
    df_list.append(df_2)
    names = ['train', 'test']
    
    # 입력한 컬럼이 DAYS_BIRTH인 경우 If로 처리
    
    if (column == 'DAYS_BIRTH') == True:    
        for dataset,name in zip(df_list,names):
            dataset['{}_year'.format(column_name)] = dataset[column].abs()//365
            dataset['{}_month'.format(column_name)] = (dataset[column].abs()%365)//30
            dataset['{}_total'.format(column_name)] = dataset['{}_year'.format(column_name)] + round(dataset['{}_month'.format(column_name)] * (1/12), 2)
            dataset.drop(columns=['{}_month'.format(column_name)], inplace=True)
        
            print('{0}set에 {1}_year, {1}_total 컬럼을 생성하였습니다. \n'.format(name,column_name))
    
    # 입력한 컬럼이 DAYS_EMPLOYED인 경우 else로 처리 (occyp_type 처리를 위한 작업이므로 1번째로 수행)
    
    else:
        for dataset,name in zip(df_list,names):
            print('{}set에 근무일자가 잘못기입된 '.format(name) +
                  '\033[96m' + '{} cases'.format(len(dataset[dataset['DAYS_EMPLOYED']>=0])) + 
                  '\033[0m' + '에 대한 처리를 시작합니다.')
            dataset['DAYS_EMPLOYED'] = dataset['DAYS_EMPLOYED'].replace(365243, 0)
            
            dataset['{}_year'.format(column_name)] = dataset[column].abs()//365
            dataset['{}_month'.format(column_name)] = (dataset[column].abs()%365)//30
            dataset['{}_total'.format(column_name)] = dataset['{}_year'.format(column_name)] + round(dataset['{}_month'.format(column_name)] * (1/12), 2)
            dataset.drop(columns=['{}_month'.format(column_name)], inplace=True)
        
            print('{0}set에 {1}_year, {1}_total 컬럼을 생성하였습니다. \n'.format(name, column_name))
        
    del df_list,names
    return dataset.head(1)

    

In [17]:
year_month_total(train, test,'DAYS_BIRTH','age')

trainset에 age_year, age_total 컬럼을 생성하였습니다. 

testset에 age_year, age_total 컬럼을 생성하였습니다. 



,gender,car,reality,child_num,income_total,income_type,edu_type,family_type,house_type,DAYS_BIRTH,DAYS_EMPLOYED,work_phone,phone,email,occyp_type,family_size,begin_month,age_year,age_total
0,M,Y,N,0,112500.0,Pensioner,Secondary / secondary special,Civil marriage,House / apartment,-21990,365243,0,1,0,NaN,2.0,-60.0,60,60.25


In [18]:
year_month_total(train, test,'DAYS_EMPLOYED','work')

trainset에 근무일자가 잘못기입된 4438 cases에 대한 처리를 시작합니다.
trainset에 work_year, work_total 컬럼을 생성하였습니다. 

testset에 근무일자가 잘못기입된 1697 cases에 대한 처리를 시작합니다.
testset에 work_year, work_total 컬럼을 생성하였습니다. 



,gender,car,reality,child_num,income_total,income_type,edu_type,family_type,house_type,DAYS_BIRTH,...,work_phone,phone,email,occyp_type,family_size,begin_month,age_year,age_total,work_year,work_total
0,M,Y,N,0,112500.0,Pensioner,Secondary / secondary special,Civil marriage,House / apartment,-21990,...,0,1,0,NaN,2.0,-60.0,60,60.25,0,0.0


### 2. 직업 Nan 값 처리

In [19]:
# 2nd

def occyp_type_nan(df_1, df_2):
    """[직업 컬럼 내의 Nan 값을 처리합니다.]

    Args:
        df_1 ([DataFrame]): [데이터 프레임 이름을 입력해주세요]
        df_2 ([DataFrame]): [데이터 프레임 이름을 입력해주세요]
    """
    # 변수 설정
    
    df_list = []
    df_list.append(df_1)
    df_list.append(df_2)
    names = ['train', 'test']

    # 무직자 처리 함수
    
    for dataset,name in zip(df_list,names):
        dataset['occyp_type'] = dataset['occyp_type'].fillna('Nan')
        
        print('직업 컬럼의 ' +
              '\033[92m' + 'Nan value {}명 중 '.format(len(dataset.loc[dataset['occyp_type'] == 'Nan'])) +
              '\033[96m' + '{}명이 실제 고용일수가 0일입니다. '.format(len(dataset.loc[(dataset['occyp_type'] == 'Nan') & (dataset['DAYS_EMPLOYED'] == 0)])) + 
              '\033[0m'+ '따라서 jobless로 처리합니다.')
        
        dataset.loc[(dataset['occyp_type'] == 'Nan') & (dataset['DAYS_EMPLOYED'] == 0), 'occyp_type'] = 'jobless'
        
        # 무직자는 아니지만 데이터가 없는 경우
        
        print('나머지 {}명 데이터는 no_data로 처리합니다.'.format(len(dataset.loc[dataset['occyp_type'] == 'Nan'])))
        
        dataset.loc[dataset['occyp_type'] == 'Nan', 'occyp_type'] = 'no data'
        
        print('{}set의 직업 Nan value를 처리하였습니다.\n'.format(name))
        
    del df_list, names
    return dataset.head(1)

occyp_type_nan(train, test)

직업 컬럼의 Nan value 8171명 중 4438명이 실제 고용일수가 0일입니다. 따라서 jobless로 처리합니다.
나머지 3733명 데이터는 no_data로 처리합니다.
trainset의 직업 Nan value를 처리하였습니다.

직업 컬럼의 Nan value 3152명 중 1697명이 실제 고용일수가 0일입니다. 따라서 jobless로 처리합니다.
나머지 1455명 데이터는 no_data로 처리합니다.
testset의 직업 Nan value를 처리하였습니다.



,gender,car,reality,child_num,income_total,income_type,edu_type,family_type,house_type,DAYS_BIRTH,...,work_phone,phone,email,occyp_type,family_size,begin_month,age_year,age_total,work_year,work_total
0,M,Y,N,0,112500.0,Pensioner,Secondary / secondary special,Civil marriage,House / apartment,-21990,...,0,1,0,jobless,2.0,-60.0,60,60.25,0,0.0


### 3. ID컬럼 생성

In [22]:
def ID_col(df_1, df_2):
    """[데이터 셋 전처리 함수입니다.]

    Args:
        df_1 ([Dataset]): [train 데이터셋을 입력해주세요]
        df_2 ([Dataset]): [test 데이터셋을 입력해주세요]
    """
    
    # 변수 설정
     
    df_list = []
    df_list.append(df_1)
    df_list.append(df_2)
    a = []
    change_credit = 0
    names = ['train', 'test']
    
    # ID 생성
    
    # for name, dataset, in zip(names, df_list):
    for dataset in df_list:
        dataset['ID'] = dataset['gender'].astype(str) +'.'+ dataset['car'].astype(str)+'.' + dataset['reality'].astype(str)+'.' + \
                dataset['child_num'].astype(str)+'.' + dataset['income_total'].astype(str) +'.'+ dataset['income_type'].astype(str) +'.'+ \
                dataset['edu_type'].astype(str)+'.' + dataset['family_type'].astype(str)+'.' + dataset['house_type'].astype(str) +'.'+ \
                dataset['DAYS_BIRTH'].astype(str) +'.'+ dataset['DAYS_EMPLOYED'].astype(str) +'.'+ dataset['work_phone'].astype(str) +'.'+ \
                dataset['phone'].astype(str) +'.'+ dataset['email'].astype(str) +'.' + dataset['family_size'].astype(str)+'.' + dataset['occyp_type'].astype(str)
        
        dataset = dataset.sort_values(['ID','begin_month'], ascending=[True, False])
        dataset.reset_index(inplace=True)
        dataset.drop(columns={'index'}, inplace=True)

        # OrdinalEncoder로 ID 변환
        
        print('\033[96m' + '해당 Data 내의 고유 ID 수는 {}입니다. '.format(dataset['ID'].nunique()) + 
              '\033[0m' + 'OrdinalEncoder를 이용힌 변환을 시작합니다.')
        encode = OrdinalEncoder()
        dataset[['ID']] = encode.fit_transform(dataset[['ID']])

        # Nth_card 컬럼 추가 (ID가 같은데 카드 개수가 여러 개인 사람을 대상으로 몇 번째 카드를 만들었는지 생성해준다)
        
        print('Column(Nth_card)을 생성합니다.')
        for i in tqdm(range(dataset['ID'].nunique())):
            Nth_card = len(dataset[dataset['ID']==i]) + 1
            
            while Nth_card > 0:
                Nth_card = Nth_card -1
                a.append(Nth_card)
                if Nth_card == 1:
                    break
                
        dataset['Nth_card'] = a        
        a.clear()
        print('\033[92m' + '신규로 카드를 개설한 사람의 수 : {}'.format(dataset[dataset['begin_month']==0.0]['ID'].nunique()) +
              '\033[0m')
        
        # Credit이 좋아지지 않고 악화된 경우를 체크하는 부분 (begin_month기준으로 Nth_card를 추가해줬으므로 인덱스 값을 기준으로 시행한다.)
        
        if (dataset['ID'].nunique() > 8000)==True:
            print('credit 정보를 확인하고 시간이 지나며 credit이 악화된 사례가 있는지 확인합니다.')
            for i in tqdm(range(dataset['ID'].nunique())):
                num = dataset.loc[dataset['ID'] == i]['credit'].to_list()
                for j in range(len(num)-1):
                    if (num[j] <= num[j+1]) == True:
                        pass
                    else :
                        change_credit += 1
                        break
            print('\033[96m'+'{}명 (전체 인원 중 {:.2f}%)은 credit이 개선되지 않고 악화된 사례가 존재합니다.'.format(change_credit, 
                                                                        (change_credit/dataset['ID'].nunique())*100) + 
                  '\033[0m')  
        else:
            print('\033[96m' + 'credit 정보가 없는 testset입니다.' + 
                  '\033[0m')
        
        # dataset 저장부분
        
        print('dataset(user_list_{})을 저장합니다. \n')    
        dataset.to_csv('{}user_list_{}.csv')
        
    
    del names, a
    return dataset.head(5)

In [26]:
train

,gender,car,reality,child_num,income_total,income_type,edu_type,family_type,house_type,DAYS_BIRTH,...,email,occyp_type,family_size,begin_month,credit,age_year,age_total,work_year,work_total,ID
0,F,N,N,0,202500.0,Commercial associate,Higher education,Married,Municipal apartment,-13899,...,0,no data,2.0,-6.0,1.0,38,38.00,12,12.83,F.N.N.0.202500.0.Commercial associate.Higher e...
1,F,N,Y,1,247500.0,Commercial associate,Secondary / secondary special,Civil marriage,House / apartment,-11380,...,1,Laborers,3.0,-5.0,1.0,31,31.17,4,4.17,F.N.Y.1.247500.0.Commercial associate.Secondar...
2,M,Y,Y,0,450000.0,Working,Higher education,Married,House / apartment,-19087,...,0,Managers,2.0,-22.0,2.0,52,52.25,12,12.08,M.Y.Y.0.450000.0.Working.Higher education.Marr...
3,F,N,Y,0,202500.0,Commercial associate,Secondary / secondary special,Married,House / apartment,-15088,...,0,Sales staff,2.0,-37.0,0.0,41,41.33,5,5.67,F.N.Y.0.202500.0.Commercial associate.Secondar...
4,F,Y,Y,0,157500.0,State servant,Higher education,Married,House / apartment,-15037,...,0,Managers,2.0,-26.0,2.0,41,41.17,5,5.75,F.Y.Y.0.157500.0.State servant.Higher educatio...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26452,F,N,N,2,225000.0,State servant,Secondary / secondary special,Married,House / apartment,-12079,...,0,Core staff,4.0,-2.0,1.0,33,33.08,5,5.42,F.N.N.2.225000.0.State servant.Secondary / sec...
26453,F,N,Y,1,180000.0,Working,Higher education,Separated,House / apartment,-15291,...,0,no data,2.0,-47.0,2.0,41,41.83,6,6.75,F.N.Y.1.180000.0.Working.Higher education.Sepa...
26454,F,Y,N,0,292500.0,Working,Secondary / secondary special,Civil marriage,With parents,-10082,...,0,Core staff,2.0,-25.0,2.0,27,27.58,5,5.50,F.Y.N.0.292500.0.Working.Secondary / secondary...
26455,M,N,Y,0,171000.0,Working,Incomplete higher,Single / not married,House / apartment,-10145,...,0,Laborers,1.0,-59.0,2.0,27,27.75,0,0.25,M.N.Y.0.171000.0.Working.Incomplete higher.Sin...


In [23]:
ID_col(train, test)

해당 Data 내의 고유 ID 수는 8759입니다. OrdinalEncoder를 이용힌 변환을 시작합니다.
Column(Nth_card)을 생성합니다.


100%|██████████| 8759/8759 [00:01<00:00, 6211.07it/s]


신규로 카드를 개설한 사람의 수 : 214
credit 정보를 확인하고 시간이 지나며 credit이 악화된 사례가 있는지 확인합니다.


100%|██████████| 8759/8759 [00:01<00:00, 4763.44it/s]


1842명 (전체 인원 중 21.03%)은 credit이 개선되지 않고 악화된 사례가 존재합니다.
dataset(user_list_{})을 저장합니다. 

해당 Data 내의 고유 ID 수는 5585입니다. OrdinalEncoder를 이용힌 변환을 시작합니다.
Column(Nth_card)을 생성합니다.


100%|██████████| 5585/5585 [00:00<00:00, 6456.43it/s]


신규로 카드를 개설한 사람의 수 : 82
credit 정보가 없는 testset입니다.
dataset(user_list_{})을 저장합니다. 



,gender,car,reality,child_num,income_total,income_type,edu_type,family_type,house_type,DAYS_BIRTH,...,email,occyp_type,family_size,begin_month,age_year,age_total,work_year,work_total,ID,Nth_card
0,F,N,N,0,103500.0,Pensioner,Higher education,Married,House / apartment,-21701,...,0,jobless,2.0,-3.0,59,59.42,0,0.0,0.0,3
1,F,N,N,0,103500.0,Pensioner,Higher education,Married,House / apartment,-21701,...,0,jobless,2.0,-46.0,59,59.42,0,0.0,0.0,2
2,F,N,N,0,103500.0,Pensioner,Higher education,Married,House / apartment,-21701,...,0,jobless,2.0,-53.0,59,59.42,0,0.0,0.0,1
3,F,N,N,0,103500.0,Pensioner,Higher education,Single / not married,House / apartment,-24543,...,0,jobless,1.0,-28.0,67,67.17,0,0.0,1.0,1
4,F,N,N,0,103500.0,Pensioner,Secondary / secondary special,Married,House / apartment,-20123,...,0,jobless,2.0,-11.0,55,55.08,0,0.0,2.0,2


In [28]:
train.to_csv('user_list_train.csv')
test.to_csv('user_list_test.csv')

In [70]:
user_list_train

NameError: name 'user_list_train' is not defined

In [29]:
train = pd.read_csv(r'C:\Users\user\Desktop\code\[유비온]신용카드 사용자 연제 예측\SH\user_list_train.csv')
test = pd.read_csv(r'C:\Users\user\Desktop\code\[유비온]신용카드 사용자 연제 예측\SH\user_list_test.csv')

### faimly_size 조정

In [30]:
def replace_value_family(df_1, df_2, num_of_outlier):
    """[Family_size의 outlier를 대치해주는 작업을 시행합니다.]

    Args:
        df_1 ([DataFrame]): [데이터 프레임 이름을 입력해주세요]
        df_2 ([DataFrame]): [데이터 프레임 이름을 입력해주세요]
        num_of_outlier ([outlier_standard]): [family_size의 outlier 기준을 입력하세요]
    """
    # 변수 설정
    
    df_list = []
    df_list.append(df_1)
    df_list.append(df_2)
    names = ['train', 'test']

    # family_size Outlier를 처리하는 작업
    
    for dataset,name in zip(df_list,names):
        
        print('family_size와 child_num의 다중공선성 문제로 child_num 컬럼을 삭제해줍니다.')
        
        dataset.drop(columns=['child_num'], inplace=True)
        
        print('\033[96m' + 'family_size {}명 이상인 사람의 수 : {}'.format(num_of_outlier, 
                                                                      len(dataset.loc[dataset['family_size'] >= num_of_outlier])) +
              '\033[0m')
        
        sub = dataset.loc[dataset['family_size']>= num_of_outlier]
        
        for i in tqdm(range(len(sub))):
        
            a = int(dataset[dataset['family_type'] == sub['family_type'].values[i]].mean()['family_size'])
            
            dataset.loc[(dataset['family_size']>= num_of_outlier), 'family_size'] = a
        
        # 잘 처리되었는지 확인하는 부분
        
        print('\033[96m' + 'family_size {}명 이상인 사람의 수 : {}'.format(num_of_outlier, 
                                                        len(dataset.loc[dataset['family_size'] >= num_of_outlier]))+
              '\033[0m')

        # family_size를 조정하고 난 후에 1인당 소득으로 total_income을 scale_down 해줌.
        
        print('1인당 소득으로 소득 수준을 조정합니다.')
        
        dataset['income_mean'] = dataset['income_total'] / dataset['family_size']
        
        print('{} 데이터 셋 처리 완료.\n'.format(name))
    
    del sub,df_list, names
    return dataset.head(1)

replace_value_family(train, test, 7)

family_size와 child_num의 다중공선성 문제로 child_num 컬럼을 삭제해줍니다.
family_size 7명 이상인 사람의 수 : 15


100%|██████████| 15/15 [00:38<00:00,  2.57s/it]


family_size 7명 이상인 사람의 수 : 0
1인당 소득으로 소득 수준을 조정합니다.
train 데이터 셋 처리 완료.

family_size와 child_num의 다중공선성 문제로 child_num 컬럼을 삭제해줍니다.
family_size 7명 이상인 사람의 수 : 10


100%|██████████| 10/10 [00:01<00:00,  7.94it/s]

family_size 7명 이상인 사람의 수 : 0
1인당 소득으로 소득 수준을 조정합니다.
test 데이터 셋 처리 완료.



,Unnamed: 0,gender,car,reality,income_total,income_type,edu_type,family_type,house_type,DAYS_BIRTH,...,email,occyp_type,family_size,begin_month,age_year,age_total,work_year,work_total,ID,income_mean
0,0,M,Y,N,112500.0,Pensioner,Secondary / secondary special,Civil marriage,House / apartment,-21990,...,0,jobless,2.0,-60.0,60,60.25,0,0.0,M.Y.N.0.112500.0.Pensioner.Secondary / seconda...,56250.0


In [40]:
test.drop(columns=['DAYS_BIRTH','DAYS_EMPLOYED'], inplace=True)

In [68]:
train

,gender,car,reality,income_total,income_type,edu_type,family_type,house_type,work_phone,phone,...,occyp_type,family_size,begin_month,credit,age_year,age_total,work_year,work_total,ID,income_mean
0,0.0,0.0,0.0,202500.0,0.0,1.0,1.0,2.0,0,0,...,19.0,2.0,-6.0,1.0,38,38.00,12,12.83,529.0,101250.0
1,0.0,0.0,1.0,247500.0,0.0,4.0,0.0,1.0,0,0,...,8.0,3.0,-5.0,1.0,31,31.17,4,4.17,3923.0,82500.0
2,1.0,1.0,1.0,450000.0,4.0,1.0,1.0,1.0,0,1,...,10.0,2.0,-22.0,2.0,52,52.25,12,12.08,8275.0,225000.0
3,0.0,0.0,1.0,202500.0,0.0,4.0,1.0,1.0,0,1,...,14.0,2.0,-37.0,0.0,41,41.33,5,5.67,2496.0,101250.0
4,0.0,1.0,1.0,157500.0,2.0,1.0,1.0,1.0,0,0,...,10.0,2.0,-26.0,2.0,41,41.17,5,5.75,4934.0,78750.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26452,0.0,0.0,0.0,225000.0,2.0,4.0,1.0,1.0,0,0,...,3.0,4.0,-2.0,1.0,33,33.08,5,5.42,1290.0,56250.0
26453,0.0,0.0,1.0,180000.0,4.0,1.0,2.0,1.0,0,0,...,19.0,2.0,-47.0,2.0,41,41.83,6,6.75,3830.0,90000.0
26454,0.0,1.0,0.0,292500.0,4.0,4.0,0.0,5.0,0,0,...,3.0,2.0,-25.0,2.0,27,27.58,5,5.50,4541.0,146250.0
26455,1.0,0.0,1.0,171000.0,4.0,2.0,3.0,1.0,0,0,...,8.0,1.0,-59.0,2.0,27,27.75,0,0.25,6372.0,171000.0


In [32]:
names = ['train', 'test']

for name,dataset in zip(names, [train, test]):
    dataset.drop(columns=['DAYS_BIRTH','DAYS_EMPLOYED'], inplace=True)
    dataset.to_csv('{}final_set_{}.csv')

KeyError: "['DAYS_BIRTH' 'DAYS_EMPLOYED'] not found in axis"

In [47]:
numerical_feats = train.dtypes[train.dtypes != "object"].index.tolist()
print("Number of Numerical features: ", len(numerical_feats))

categorical_feats = train.dtypes[train.dtypes == "object"].index.tolist()
print("Number of Categorical features: ", len(categorical_feats))

Number of Numerical features:  12
Number of Categorical features:  9


In [48]:
#train object 인코딩
from sklearn.preprocessing import OrdinalEncoder
encoder = OrdinalEncoder()
train[categorical_feats] = encoder.fit_transform(train[categorical_feats], train['credit'])
#test 인코딩
test[categorical_feats] = encoder.fit_transform(test[categorical_feats])

# MODELING

In [51]:
#train, test x,y 설정
train_x=train.drop('credit', axis=1)
train_y=train[['credit']]
test_x=test
print(train_x.shape, train_y.shape, test.shape)

(26457, 20) (26457, 1) (10000, 20)


In [58]:
#Random Forest model fit
from sklearn.ensemble import RandomForestClassifier #분류트리(모델) 
from sklearn.model_selection import train_test_split # train/test
from sklearn.metrics import accuracy_score, confusion_matrix # 평가 : 분류정확도 
from sklearn.metrics import classification_report # 평가 : 정확률, 재현율, f1_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, log_loss
from sklearn.metrics import f1_score
from tensorflow.keras.utils import to_categorical

X_train, X_val, y_train, y_val = train_test_split(train_x, train_y, 
                                                    stratify=train_y, test_size=0.25,
                                                    random_state = 10086)

print("Train set: ")
print(X_train.shape)
print(y_train.shape)
print("===========")
print("Validation set: ")
print(X_val.shape)
print(y_val.shape)


clf=RandomForestClassifier()
clf.fit(X_train, y_train)
y_pred=clf.predict_proba(X_val)

print(f"log_loss: {log_loss(to_categorical(y_val['credit']), y_pred)}")

Train set: 
(19842, 20)
(19842, 1)
Validation set: 
(6615, 20)
(6615, 1)
log_loss: 1.0029295122124835


In [60]:
#Run KFOLD
from sklearn.model_selection import KFold, StratifiedKFold

def run_kfold(clf):
    folds=StratifiedKFold(n_splits=5, shuffle=True, random_state=55)
    outcomes=[]
    sub=np.zeros((test_x.shape[0], 3))  
    for n_fold, (train_index, val_index) in enumerate(folds.split(train_x, train_y)):
        X_train, X_val = train_x.iloc[train_index], train_x.iloc[val_index]
        y_train, y_val = train_y.iloc[train_index], train_y.iloc[val_index]
        clf.fit(X_train, y_train)
        
        predictions=clf.predict_proba(X_val)
        
        logloss=log_loss(to_categorical(y_val['credit']), predictions)
        outcomes.append(logloss)
        print(f"FOLD {n_fold} : logloss:{logloss}")
        
        sub+=clf.predict_proba(test_x)
        
        
    mean_outcome=np.mean(outcomes)
    
    print("Mean:{}".format(mean_outcome))
    return sub/folds.n_splits

my_submission = run_kfold(clf)

FOLD 0 : logloss:1.0546674343355116
FOLD 1 : logloss:1.0412794313059484
FOLD 2 : logloss:0.970839060351301
FOLD 3 : logloss:0.9357214857572841
FOLD 4 : logloss:1.0646643933042552
Mean:1.01343436101086


In [62]:
my_submission

array([[0.0405    , 0.228     , 0.7315    ],
       [0.382     , 0.197     , 0.421     ],
       [0.0575    , 0.051     , 0.8915    ],
       ...,
       [0.0428    , 0.09253333, 0.86466667],
       [0.351     , 0.278     , 0.371     ],
       [0.13766667, 0.25833333, 0.604     ]])

In [63]:
submission.loc[:,1:]=my_submission

In [64]:
submission

,index,0,1,2
0,26457,0.040500,0.228000,0.731500
1,26458,0.382000,0.197000,0.421000
2,26459,0.057500,0.051000,0.891500
3,26460,0.106200,0.095500,0.798300
4,26461,0.085500,0.205500,0.709000
...,...,...,...,...
9995,36452,0.112000,0.258733,0.629267
9996,36453,0.201333,0.396333,0.402333
9997,36454,0.042800,0.092533,0.864667
9998,36455,0.351000,0.278000,0.371000


In [67]:
submission.to_csv('1102_non_scale.csv', index = False)